# MA 195L.2 Project 2: Forward Rate Agreements

## Version 1: May 19, 2021

### Instructor: Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MA 195L.2: Introduction to Financial Mathematics II \
Second Semester, S.Y. 2020-2021 \
Ateneo de Manila University

Submitted by: Jakov Ivan S. Dumbrique



# Part 1: Functional Implementation

The specifications of the functions are specified in the succeeding code blocks. You may create helper functions you can call for reusing computations.

In [1]:
def linear_interp(t,x,y):
    m = (y[1]-y[0])/(x[1]-x[0])
    y = y[0] + m*(t-x[0])
    return y

In [2]:
def forward_rate(
    t, T_1, T_2, r_1, r_2, m 
):
    """Returns the forward rate observed at time t (0<=t<T_1) applicable to the period [T_1, T_2] 
    (where T_1 < T_2 and their values are determined at time 0)
    
       This function assumes that the zero rates r_1 and r_2 and the forward rate 
       have the same compounding frequency. 
    
    Parameters
    ----------
    t (in years)
    T_1: start of forward period as observed at time t=0 (in years)
    T_2: end of forward period as observed at time t=0 (in years)
    r_1: zero rate observed at time t with tenor T_1-t years
    r_2: zero rate observed at time t with tenor T_2-t years
    m: compounding frequency of r_1, r_2 and the forward rate 
        assumes value in this list: ['simple', 'exp', 1, 2, 4, 12, 52, 365]
    
    Returns
    -------
    float (4 decimal places)
        forward rate observed at time t (0<=t<T_1) applicable to the period [T_1, T_2]
    """
    
    if m == "simple":
        return ((1 + r_2 * T_2 ) / (1 + r_1 * T_1) - 1) / (T_2 - T_1)
    elif m == "exp":
        return (r_2 * T_2  - r_1 * T_1) / (T_2 - T_1)
    else:
        return (((1 + r_2/m) ** (m * T_2) * (1 + r_1/m) ** (-m * T_1)) ** (1/(m*(T_2-T_1))) - 1 ) * m

In [3]:
import numpy as np

def fra_value(
    t, T_1, T_2, r_fixed, r_float, r_t, m, L, payer 
):
    """Returns the value of the forward rate agreement (FRA) at time t (0<t<=T_1) to the specified payer
    
        at time t=T_1, the function returns the present value (at time t=T_1) of the FRA's cash settlement 
        from the perspective of the specified payer
        
        This function assumes that the interest rates r_fixed, r_float, and r_t 
        have the same compounding frequency. 
    
    
    Parameters
    ----------
    t (in years) 
    T_1: start of forward period as observed at time t=0 (in years)
    T_2: end of forward period as observed at time t=0 (in years)
    r_fixed: fixed interest rate of the FRA determined at time t=0
    r_float: floating interest rate of the FRA determined at time t
    r_t: zero rate observed at time t with tenor T_2-t years
    m: compounding frequency of r_1, r_2 and the forward rate 
        assumes value in this list: ['simple', 'exp', 1, 2, 4, 12, 52, 365]
    L: notional principal amount of the FRA
    payer: we are getting the value of the FRA from the perspective of the payer
        assumes value in this list: ['fixed', 'float']
    
    Returns
    -------
    float (2 decimal places)
        value of the forward rate agreement (FRA) at time t (0<t<=T_1) to the specified payer
    """
        
    if m == "simple":
        value = L * (r_float - r_fixed) * (T_2 - T_1) / (1 + r_t * (T_2 - t))
    elif m == "exp":
        value = L * (np.exp(r_float * (T_2 - T_1)) - np.exp(r_fixed * (T_2 - T_1))) * np.exp(-r_t * (T_2 - t))
    else:
        value = L * ((1 + r_float/m) ** (m * (T_2 - T_1)) - (1 + r_fixed/m) ** (m * (T_2 - T_1))) * (1 + r_t/m) ** (-m * (T_2 - t))
    
    if payer == "fixed":
        return value
    else:
        return -value

# Part 2: Validation
In Part 2, you will be using the functions you created in Part 1 to answer the following questions.

## Question A

Given the following zero rates: 1-month: 1.75%, 3-month:2.5%, 6-month: 3.75%, 9-month: 4.25%, 1-year: 4.75%.

1. If the given zero rates are simple rates, calculate the 3-month forward rate 3 months from now.
2. If the given zero rates are compounded annually, calculate the 5-month forward rate 1 month from now.
3. If the given zero rates are compounded semi-annually, calculate the 6-month forward rate 6 months from now.
4. If the given zero rates are compounded monthly, calculate the 6-month forward rate 3 months from now.
5. If the given zero rates are continuously-compounded, calculate the 3-month forward rate 6 months from now.

In [4]:
print("1. The forward rate is {}.".format(round(forward_rate(0, 3/12, 6/12, 0.025, 0.0375, "simple"),4)))
print("2. The forward rate is {}.".format(round(forward_rate(0, 1/12, 6/12, 0.0175, 0.0375, 1),4)))
print("3. The forward rate is {}.".format(round(forward_rate(0, 6/12, 1, 0.0375, 0.0475, 2),4)))
print("4. The forward rate is {}.".format(round(forward_rate(0, 3/12, 9/12, 0.025, 0.0425, 12),4)))
print("5. The forward rate is {}.".format(round(forward_rate(0, 6/12, 9/12, 0.0375, 0.0425, "exp"),4)))

1. The forward rate is 0.0497.
2. The forward rate is 0.0415.
3. The forward rate is 0.0575.
4. The forward rate is 0.0513.
5. The forward rate is 0.0525.


## Question B

Suppose that a $(3 \times 6)$ FRA is set today with notional 15 million pesos. The 3-month rate now with continuous compounding is 2.250% and the 6-month rate with continuous compounding is 3.175%.

1. Determine the fixed rate for the FRA.
2. After 1 month, the 2-month rate is 2.125% and the 5-month rate is 3.425%, both with continuous compounding. What is the value of the FRA to the fixed-rate payer? to the floating-rate payer?
3. Suppose the 3-month rate at expiration date of the FRA is 3.915% with continuous compounding. Which party pays the other party? How much is the settlement amount?

In [5]:
r_fixed = forward_rate(0, 3/12, 6/12, 0.0225, 0.03175, "exp")
print("1. The fixed rate of the FRA is {}.".format(round(r_fixed,4)))

fra = fra_value(1/12, 3/12, 6/12, r_fixed, 0.03425, 0.02125, "exp", 15000000, "fixed")
print("2. The value of the FRA to the fixed-rate payer is Php {}.".format(round(fra,2)))
print("2. The value of the FRA to the floating-rate payer is Php {}.".format(round(-fra,2)))

fra = fra_value(3/12, 3/12, 6/12, r_fixed, 0.03915, 0.03915, "exp", 15000000, "float")
print("3. The fixed-rate payer pays the floating-rate payer Php {}.".format(round(fra,2)))

1. The fixed rate of the FRA is 0.041.
2. The value of the FRA to the fixed-rate payer is Php -25326.48.
2. The value of the FRA to the floating-rate payer is Php 25326.48.
3. The fixed-rate payer pays the floating-rate payer Php 6939.1.


## Question C

A $(6 \times 12)$ FRA is set today with notional 20 million pesos. The 6-month zero rate with semi-annual compounding is 3.875% and the 1-year interest rate with semi-annual compounding is 4.125%.

1. Determine the fixed rate for the FRA.
2. After 3 months, the 3-month zero rate with quarterly compounding is 3.225% and the 9-month zero rate with quarterly compounding is 3.725%. What is the value of the FRA to the fixed-rate payer? to the floating-rate payer?
3. Suppose the 6-month zero rate with semi-annual compounding at expiration date of the FRA is 4.425%. Which party pays the other party? How much is the settlement amount?

In [6]:
r_fixed = forward_rate(0, 6/12, 1, 0.03875, 0.04125, 2)
print("1. The fixed rate of the FRA is {}.".format(round(r_fixed,4)))

# Convert from quarterly to semi-annually
r_float = ( (1+0.03725/4) ** ((4*9/12)/(2*9/12)) -1 ) * 2 
r_t = ( (1+0.03225/4) ** ((4*3/12)/(2*3/12)) -1 ) * 2 
fra = fra_value(3/12, 6/12, 1, r_fixed, r_float, r_t, 2, 20000000, "fixed")
print("2. The value of the FRA to the fixed-rate payer is Php {}.".format(round(fra,2)))
print("2. The value of the FRA to the floating-rate payer is Php {}.".format(round(-fra,2)))

fra = fra_value(6/12, 6/12, 1, r_fixed, 0.04425, 0.04425, 2, 20000000, "fixed")
print("3. The floating-rate payer pays the fixed-rate payer Php {}.".format(round(fra,2)))

1. The fixed rate of the FRA is 0.0438.
2. The value of the FRA to the fixed-rate payer is Php -61789.59.
2. The value of the FRA to the floating-rate payer is Php 61789.59.
3. The floating-rate payer pays the fixed-rate payer Php 4861.78.


## Question D

A $(6 \times 18)$ FRA with notional 5 million pesos was set 2 months ago for a fixed simple rate of 6.725%. Suppose the simple zero rates now are as follows.

<table>
    <tr>
        <td>
           tenor
        </td>
        <td>
            1 month
        </td>
        <td>
            3 months
        </td>
        <td>
            6 months
        </td>
        <td>
            1 year
        </td>
        <td>
            2 years
        </td>
    </tr>
        <tr>
        <td>
            interest rate
        </td>
        <td>
            4.205%
        </td>
        <td>
            4.515%
        </td>
        <td>
            4.815%
        </td>
        <td>
            6.235%
        </td>
        <td>
            6.925%
        </td>
    </tr>
</table>

1. Use linear interpolation to estimate the 4-month and 16-month zero rates.
2. Determine the value of the FRA now to the fixed-rate payer.
3. After 2 months (from now), the simple zero rates are as follows. What is the value of the FRA to the floating-rate payer?

<table>
    <tr>
        <td>
           tenor
        </td>
        <td>
            1 month
        </td>
        <td>
            3 months
        </td>
        <td>
            6 months
        </td>
        <td>
            1 year
        </td>
        <td>
            2 years
        </td>
    </tr>
        <tr>
        <td>
            interest rate
        </td>
        <td>
            4.125%
        </td>
        <td>
            4.405%
        </td>
        <td>
            5.015%
        </td>
        <td>
            5.925%
        </td>
        <td>
            6.215%
        </td>
    </tr>
</table>

 
4. At expiration date, the simple zero rates are as follows. Which party pays the other party? How much is the settlement amount?

<table>
    <tr>
        <td>
           tenor
        </td>
        <td>
            1 month
        </td>
        <td>
            3 months
        </td>
        <td>
            6 months
        </td>
        <td>
            1 year
        </td>
        <td>
            2 years
        </td>
    </tr>
        <tr>
        <td>
            interest rate
        </td>
        <td>
            4.165%
        </td>
        <td>
            4.455%
        </td>
        <td>
            4.915%
        </td>
        <td>
            6.075%
        </td>
        <td>
            6.565%
        </td>
    </tr>
</table>

In [7]:
r_float = linear_interp(16/12,[1,2],[0.06235,0.06925])
r_t = linear_interp(4/12,[3/12,6/12],[0.04515,0.04815])
print("1. The 4-month zero rate is {}.".format(round(r_t,4)))
print("1. The 16-month zero rate is {}.".format(round(r_float,4)))

r_fixed = 0.06725
fra = fra_value(2/12, 6/12, 18/12, r_fixed, r_float, r_t, "simple", 5000000, "fixed")
print("2. The value of the FRA to the fixed-rate payer is Php {}.".format(round(fra,2)))

r_float = linear_interp(14/12,[1,2],[0.05925,0.06215])
r_t = linear_interp(2/12,[1/12,3/12],[0.04125,0.04405])
fra = fra_value(4/12, 6/12, 18/12, r_fixed, r_float, r_t, "simple", 5000000, "float")
print("3. The value of the FRA to the floating-rate payer is Php {}.".format(round(fra,2)))

r_float = 0.06075
r_t = 0.06075
fra = fra_value(6/12, 6/12, 18/12, r_fixed, r_float, r_t, "simple", 5000000, "float")
print("4. At the expiration date, the fixed-rate payer pays the floating-rate payer Php {}.".format(round(fra,2)))

1. The 4-month zero rate is 0.0462.
1. The 16-month zero rate is 0.0646.
2. The value of the FRA to the fixed-rate payer is Php -12246.44.
3. The value of the FRA to the floating-rate payer is Php 35801.89.
4. At the expiration date, the fixed-rate payer pays the floating-rate payer Php 30638.7.
